In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY,
        "X-Goog-Api-Key": GOOGLE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
	)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__
live = client.is_live()
connected = client.is_connected()

print(f"Weaviate Ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"Weaviate Live: {client.is_live()}")
print(f"Client Connected: {connected}")

In [ ]:
# Change configuration for multiple collections
from weaviate.classes.config import Reconfigure, ReplicationDeletionStrategy

def get_collection_names():
    collections = [
        '<COL>',
        '<COL>',
        '<COL>'
    ]
    return collections

# Function to update the configuration for all collections
def update_collections_config(client):
    collections = get_collection_names()
    for collection_name in collections:
        try:
            # Get the collection by name
            collection = client.collections.get(collection_name)
            # Update its configuration
            collection.config.update(
                replication_config=Reconfigure.replication(
                    deletion_strategy=ReplicationDeletionStrategy.TIME_BASED_RESOLUTION,
                    async_enabled=True
                )
            )
            print(f"Updated configuration for collection: {collection_name}")
        
        except Exception as e:
            print(f"Failed to update configuration for collection {collection_name}: {str(e)}")

update_collections_config(client)

In [ ]:
# Update shards status
coll = client.collections.get("<COLLECTION_NAME>")

shards = coll.config.update_shards(
    status="READY",
    shard_names=["<SHARDS>"]  # The names (List[str]) of the shard to update (or a shard name)
)

print(shards)

In [ ]:
# Update the vector index configuration for a collection to increase the threshold
from weaviate.classes.config import Reconfigure

collection = client.collections.get("<COLLECTION_NAME>")
result = collection.config.update(
    vectorizer_config=Reconfigure.VectorIndex.dynamic(
        threshold=100000
    )
)
print(result)

In [ ]:
# Function to update object properties
def update_object_properties(client, collection_name, uuid, properties, tenant=None):
    try:
        collection = client.collections.get(collection_name)
        if tenant:
            collection = collection.with_tenant(tenant)
        
        collection.data.update(
            uuid=uuid,
            properties=properties
        )
        return True
    except Exception as e:
        raise Exception(f"Failed to update object: {str(e)}")


collection_name = "<COLLECTION_NAME>"
object_uuid = "<OBJECT_UUID>"
new_properties = {
    "<PROPERTY_NAME>": [
    { "<SUB_PROPERTY_NAME>": "<SUB_PROPERTY_VALUE>" },
    { "<SUB_PROPERTY_NAME>": "SUB_PROPERTY_VALUE" }
    ]
}

# Calling the method to update the object
result = update_object_properties(client, collection_name, object_uuid, new_properties)
print("Update successful:", result)

In [ ]:
# Update Collection Description
from weaviate.classes.config import Reconfigure

# Get the Article collection object
articles = client.collections.get("<COLLECTION_NAME>")

# Update the collection configuration to remove stopwords "a" and "the"
articles.config.update(
    description="<NEW_DESCRIPTION>",
)

In [ ]:
# Update the collection Inverted Index Configuration
from weaviate.classes.config import Reconfigure, StopwordsPreset

def update_inverted_index_config(
    client,
    collection_name,
    bm25_b=None,
    bm25_k1=None,
    cleanup_interval_seconds=None,
    stopwords_preset=None,
    stopwords_additions=None,
    stopwords_removals=None
):
    collection = client.collections.get(collection_name)
    collection.config.update(
        inverted_index_config=Reconfigure.inverted_index(
            bm25_b=bm25_b,
            bm25_k1=bm25_k1,
            cleanup_interval_seconds=cleanup_interval_seconds,
            stopwords_preset=stopwords_preset,
            stopwords_additions=stopwords_additions,
            stopwords_removals=stopwords_removals
        )
    )

update_inverted_index_config(
    client,
    "<COLLECTION_NAME>",
    bm25_b=0.5,
    bm25_k1=1.8,
    cleanup_interval_seconds=90,
    stopwords_preset=StopwordsPreset.EN,
    stopwords_additions=["<NEW_STOPWORD_1>", "<NEW_STOPWORD_2>", "<NEW_STOPWORD_3>"],
    stopwords_removals=["<STOPWORD_1>", "<STOPWORD_2>"]
)

In [ ]:
# Function to update Collection Multi-Tenancy and Replication Configurations
from weaviate.classes.config import Reconfigure, ReplicationDeletionStrategy

def update_collection_configs(
	client,
	collection_name,
	auto_tenant_creation=None,
	auto_tenant_activation=None,
	async_enabled=None,
	deletion_strategy=None
):
	collection = client.collections.get(collection_name)
	collection.config.update(
		multi_tenancy_config=Reconfigure.multi_tenancy(
			auto_tenant_creation=auto_tenant_creation,
			auto_tenant_activation=auto_tenant_activation
		),
		replication_config=Reconfigure.replication(
			async_enabled=async_enabled,
			deletion_strategy=deletion_strategy
		)
	)

update_collection_configs(
	client,
	"<COLLECTION_NAME>",
	auto_tenant_creation=True,
	auto_tenant_activation=True,
	async_enabled=True,
	deletion_strategy=ReplicationDeletionStrategy.TIME_BASED_RESOLUTION
)

In [ ]:
# Function to update collection vector index configuration
from weaviate.classes.config import Reconfigure, PQEncoderType, PQEncoderDistribution, VectorFilterStrategy

def update_vectorizer_config(
    client,
    collection_name,
    dynamic_ef_factor=None,
    dynamic_ef_min=None,
    dynamic_ef_max=None,
    filter_strategy=None,
    flat_search_cutoff=None,
    vector_cache_max_objects=None,
    pq_enabled=None,
    pq_centroids=None,
    pq_segments=None,
    pq_training_limit=None,
    pq_encoder_type=None,
    pq_encoder_distribution=None
):
    pq_config = None
    if any([pq_enabled, pq_centroids, pq_segments, pq_training_limit, pq_encoder_type, pq_encoder_distribution]):
        pq_config = Reconfigure.VectorIndex.Quantizer.pq(
            enabled=pq_enabled,
            centroids=pq_centroids,
            segments=pq_segments,
            training_limit=pq_training_limit,
            encoder_type=pq_encoder_type,
            encoder_distribution=pq_encoder_distribution
        )
    collection = client.collections.get(collection_name)
    collection.config.update(
        vectorizer_config=Reconfigure.VectorIndex.hnsw(
            dynamic_ef_factor=dynamic_ef_factor,
            dynamic_ef_min=dynamic_ef_min,
            dynamic_ef_max=dynamic_ef_max,
            filter_strategy=filter_strategy,
            flat_search_cutoff=flat_search_cutoff,
            vector_cache_max_objects=vector_cache_max_objects,
            quantizer=pq_config
        )
    )

update_vectorizer_config(
    client,
    "<COLLECTION_NAME>",
    dynamic_ef_factor=10,
    dynamic_ef_min=100,
    dynamic_ef_max=500,
    filter_strategy=VectorFilterStrategy.SWEEPING,
    flat_search_cutoff=20000,
    vector_cache_max_objects=1000000,
    pq_enabled=False,
    pq_centroids=256,
    pq_segments=96,
    pq_training_limit=50000,
    pq_encoder_type=PQEncoderType.KMEANS,
    pq_encoder_distribution=PQEncoderDistribution.NORMAL
)